# Fundamental concepts of tensorflow
## Content:
1. Introduction to tensors
2. Getting information from tensors
3. Manipulating tensors
4. Tensors & Numpy
5. Using @tf.function (a way to speed up regular python functions)
6. Using GPUs with tensorflow
7. Excercise !

1. Introduction to tensors

In [32]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
print(tf.__version__)

2.5.0


In [33]:
#creating tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [34]:
#checking number of dimensions of a tensor (ndim)
scalar.ndim


0

In [35]:
#create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10])>

In [36]:
#check dimension of vector
vector.ndim

1

In [37]:
#creating matrix (having nore than 1D)
matrix = tf.constant([[10,7],[7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]])>

In [38]:
matrix.ndim

2

In [39]:
#creating another matrix
another_matrix = tf.constant([[10.,7.],[8.,2.],[3.,9.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 8.,  2.],
       [ 3.,  9.]], dtype=float16)>

In [40]:
another_matrix.ndim

2

In [41]:
#creating tensor
tensor = tf.constant([[[1,2,3],[3,4,5]],[[4,5,6],[7,8,9]],[[10,11,12],[13,14,15]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 3,  4,  5]],

       [[ 4,  5,  6],
        [ 7,  8,  9]],

       [[10, 11, 12],
        [13, 14, 15]]])>

In [42]:
tensor.ndim

3

#Summary
1. Scalar is a single number
2. Vector is a number with direction (eg: wind speed & direction)
3. Matrix is 2D array of numbers
4. Tensor is n-dimensional array of numbers

In [43]:
##Creating tensors with tf.Variable
tf.Variable

tensorflow.python.ops.variables.Variable

In [44]:
#creating same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor,unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7])>)

In [45]:
#Let's try changing one of the elements in our changeable tensor
changeable_tensor[0] = 7

TypeError: 'ResourceVariable' object does not support item assignment

In [46]:
#trying .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7])>

In [47]:
#Let's try changing unchangeable tensor
unchangeable_tensor[0].assign(7)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

#Creating random tensors

Random tensors are tensors of arbitrary size containing random numbers

In [48]:
#creating two random but same tensors
random_1 = tf.random.Generator.from_seed(7) #set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3, 2))
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.2878567 ],
        [-0.8757901 , -0.08857017],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.2878567 ],
        [-0.8757901 , -0.08857017],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [49]:
#shuffle a tensor
not_shuffled = tf.constant([[10,7],[3,4],[2,5]])
#shuffle the non-shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [10,  7],
       [ 2,  5]])>

**shuffle doubtful (random seed generation)** -- Clarify from tensorflow doc (global level random seed/operational level random seed)
https://www.tensorflow.org/api_docs/python/tf/random/set_seed

It looks like if we want our shuffled tensors to be in the same order we got to use global level random seed as well as operation level random seed:

4. If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence.

In [50]:
#global level random seed
tf.random.set_seed(7)
#operation level random seed
tf.random.shuffle(not_shuffled, seed=7)
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

Other ways to make tensors

In [51]:
#create a tensor of all ones
tf.ones([10,7]) #defaulted to float32

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [52]:
#create a tensor of all zeros
tf.zeros([10,7]) #defaulted to float32

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

###Turm NumPy arrays into tensors

The main difference between NumPy arrays and tensors is that tensors can be run on a GPU (much faster for normal computing).

In [53]:
#turning NumPy array to tensor
import numpy as np
array_A = np.arange(1,25, dtype=np.int32) #creating a numpy array between 1 and 25
array_A
# X = tf.constant(some_matrix) #capital for matrix or tensor
# y = tf.constant(vector) #non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [54]:
A = tf.constant(array_A, shape=(3,4,2))
B = tf.constant(array_A, shape=(2,12))
C = tf.constant(array_A)
A,B,C

(<tf.Tensor: shape=(3, 4, 2), dtype=int32, numpy=
 array([[[ 1,  2],
         [ 3,  4],
         [ 5,  6],
         [ 7,  8]],
 
        [[ 9, 10],
         [11, 12],
         [13, 14],
         [15, 16]],
 
        [[17, 18],
         [19, 20],
         [21, 22],
         [23, 24]]])>,
 <tf.Tensor: shape=(2, 12), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]])>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24])>)

In [55]:
A.ndim, B.ndim, C.ndim

(3, 2, 1)

Getting information from tensors

Attributes:
1. Shape - The length (number of elements) of each of the dimensions of a tensor - tensor.shape
2. Rank - The number of tensor dimensions - tensor.ndim
3. Axis or dimension - A particular dimension of a tensor - tensor[0], tensor[:,1]..
4. Size - The total number of items in the tensor - tf.size(tensor)

In [56]:
#Creating a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2,3,4,5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [57]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [58]:
#getting various attributes of tensor
print("Datatype of every element: ",rank_4_tensor.dtype)
print("Number of dimensions (rank): ",rank_4_tensor.ndim)
print("Shape of tensor: ",rank_4_tensor.shape)
print("Elements along the 0 axis: ",rank_4_tensor.shape[0])
print("Elements along the last axis: ",rank_4_tensor.shape[-1])
print("Total number of elements in the tensor: ",tf.size(rank_4_tensor))
print("Total number of elements in the tensor: ",tf.size(rank_4_tensor).numpy())

Datatype of every element:  <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in the tensor:  tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in the tensor:  120


###Indexing tensors

Tensors can be indexed just like python lists

In [59]:
some_matrix = [1,2,3,4,5,6]
some_matrix[:3]


[1, 2, 3]

In [60]:
#getting the first 3 elements of each dimension
rank_4_tensor[:3, :3, :3, :3]

<tf.Tensor: shape=(2, 3, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [61]:
#getting first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [62]:
#create a rank 2 tensor
rank_2_tensor = tf.constant([[10,7],[3,4]])
rank_2_tensor.ndim, rank_2_tensor.shape, tf.size(rank_2_tensor).numpy()

(2, TensorShape([2, 2]), 4)

In [63]:
#getting last item of each row of the rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4])>

In [64]:
#adding an extra dimension to rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

In [65]:
#alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) #"-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

In [66]:
#expand the 0-axis
tf.expand_dims(rank_2_tensor, axis=0) #expand the 0-axis

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]])>

In [67]:
rank_2_tensor 

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]])>

Manipulating tensors

**Basic operations**

+, -, *, /

In [68]:
#adding values to tensor by addition operator
tensor = tf.constant([[10,7],[3,4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]])>

In [69]:
#but original tensor remains unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]])>

In [70]:
#multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]])>

In [71]:
#division
tensor / 10

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1. , 0.7],
       [0.3, 0.4]])>

In [72]:
#substraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]])>

In [73]:
#tensorflow inbuilt functions
tf.multiply(tensor,10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]])>

**Matrix Multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations

In [74]:
#matrix multiplication in tensorflow
print(tensor)
tf.matmul(tensor,tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]])>

In [75]:
#matrix multiplication with python @ operator
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]])>

In [76]:
#creating a 3x2 tensor
X = tf.constant([[1,2],[3,4],[5,6]])
#creating another 3x2 tensor
Y = tf.constant([[7,8],[9,10],[11,12]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]])>)

In [77]:
#changing shape of Y and multiplying
tf.matmul(X, tf.reshape(Y, shape=[2,3]))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

In [78]:
X.shape, tf.reshape(Y, shape=[2,3]).shape

(TensorShape([3, 2]), TensorShape([2, 3]))

In [79]:
#transpose
X, tf.transpose(X), tf.reshape(X, shape=[2,3])

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]])>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]])>)

**The dot product**

Matrix multiplication is also known as dot product.

Matrix multiplication can be performed using:
1. tf.tensordot()
2. tf.matmul()
3. @

In [80]:
#performing dot product on X
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]])>

Generally, when performing matrix multiplication on two tensors and one of the axes doesn't line up, we will transpose rather than reshape to satisfy multiplication rules.

**Changing the Data type of tensor**

In [81]:
#creating new tensor with default datatype (floatt32)
B = tf.constant([1.1,2.2])
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.1, 2.2], dtype=float32)>,
 tf.float32)

In [82]:
C = tf.constant([1,2])
C, C.dtype

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2])>, tf.int32)

In [83]:
#changing from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.1, 2.2], dtype=float16)>,
 tf.float16)

In [84]:
#changing from int32 to float16
E = tf.cast(C, dtype=tf.float16)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1., 2.], dtype=float16)>,
 tf.float16)

**Aggregating tensors**

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [85]:
#getting absolute values
F = tf.constant([-7,-10])
F, tf.abs(F)

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10])>)

Let's go through the following form of aggregation:

1. Get the minimum
2. Get the maximum
3. Get the mean of a tensor
4. Get the sum of a tensor

In [86]:
#creating a large tensor with vales between 0 and 100 of size 50
tensor_agg = tf.constant(np.random.randint(0,100, size=50))
tensor_agg.shape, tf.size(tensor_agg), tensor_agg.ndim, tensor_agg


(TensorShape([50]),
 <tf.Tensor: shape=(), dtype=int32, numpy=50>,
 1,
 <tf.Tensor: shape=(50,), dtype=int32, numpy=
 array([ 3, 95, 58,  9,  4,  3, 60, 68, 15, 97, 26,  6, 14, 98,  6, 68, 26,
        89, 98, 94, 46, 28,  0, 77, 37, 85, 80, 38, 28, 13, 81, 99, 95, 65,
        84, 86, 43, 88, 67, 14, 33, 50, 22, 57,  1, 14, 51, 69, 80, 20])>)

In [87]:
#finding minimum
tf.reduce_min(tensor_agg)

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [88]:
#finding max
tf.reduce_max(tensor_agg)

<tf.Tensor: shape=(), dtype=int32, numpy=99>

In [89]:
#finding mean
tf.reduce_mean(tensor_agg)

<tf.Tensor: shape=(), dtype=int32, numpy=49>

In [90]:
#finding sum
tf.reduce_sum(tensor_agg)

<tf.Tensor: shape=(), dtype=int32, numpy=2488>

In [91]:
#finding variance - to find the variance of our tensor we need access to tensorflow-probability
tfp.stats.variance(tensor_agg)

<tf.Tensor: shape=(), dtype=int32, numpy=1100>

In [92]:
#finding standard deviation - int datatype tensor won't work
tfp.stats.stddev(tf.cast(tensor_agg, dtype=tf.float16))

<tf.Tensor: shape=(), dtype=float16, numpy=33.16>

### finding positional maximum and minimum

In [116]:
#creating a new tensor to find out positional maximum and minimum
tf.random.set_seed(40)
tensor_rand = tf.random.uniform(shape=[50])
print(tensor_rand)
#finding positional maximum
print(tf.argmax(tensor_rand), tensor_rand[tf.argmax(tensor_rand)], tf.reduce_max(tensor_rand))
#finding position min
print(tf.argmin(tensor_rand), tensor_rand[tf.argmin(tensor_rand)], tf.reduce_min(tensor_rand))

tf.Tensor(
[0.54513085 0.7588961  0.8362565  0.39474642 0.72001076 0.6402768
 0.15894496 0.57128036 0.41122293 0.18993318 0.2312665  0.7875649
 0.18806243 0.8033031  0.12747478 0.43258965 0.98803484 0.8451586
 0.9797807  0.9952853  0.6283821  0.8643936  0.9500631  0.41532683
 0.388798   0.33959448 0.36432612 0.79217136 0.90930283 0.6459546
 0.4547919  0.3669002  0.22951639 0.54661405 0.9371824  0.04014015
 0.82825315 0.45409238 0.943413   0.20978856 0.1274836  0.9666685
 0.97015405 0.568432   0.99477863 0.39956594 0.3263564  0.61465824
 0.24279046 0.03102553], shape=(50,), dtype=float32)
tf.Tensor(19, shape=(), dtype=int64) tf.Tensor(0.9952853, shape=(), dtype=float32) tf.Tensor(0.9952853, shape=(), dtype=float32)
tf.Tensor(49, shape=(), dtype=int64) tf.Tensor(0.031025529, shape=(), dtype=float32) tf.Tensor(0.031025529, shape=(), dtype=float32)


####Squeezing a tensor

Removing all single dimensions

In [124]:
tf.random.set_seed(42)
tensor_sq = tf.constant(tf.random.uniform(shape=[50]), shape=(1,1,1,1,50))
print(tensor_sq, tensor_sq.shape, tensor_sq.shape)
tensor_squeezed = tf.squeeze(tensor_sq)
print(tensor_squeezed, tensor_squeezed.shape, tensor_squeezed.shape)

tf.Tensor(
[[[[[0.6645621  0.44100678 0.3528825  0.46448255 0.03366041 0.68467236
     0.74011743 0.8724445  0.22632635 0.22319686 0.3103881  0.7223358
     0.13318717 0.5480639  0.5746088  0.8996835  0.00946367 0.5212307
     0.6345445  0.1993283  0.72942245 0.54583454 0.10756552 0.6767061
     0.6602763  0.33695042 0.60141766 0.21062577 0.8527372  0.44062173
     0.9485276  0.23752594 0.81179297 0.5263394  0.494308   0.21612847
     0.8457197  0.8718841  0.3083862  0.6868038  0.23764038 0.7817228
     0.9671384  0.06870162 0.79873943 0.66028714 0.5871513  0.16461694
     0.7381023  0.32054043]]]]], shape=(1, 1, 1, 1, 50), dtype=float32) (1, 1, 1, 1, 50) (1, 1, 1, 1, 50)
tf.Tensor(
[0.6645621  0.44100678 0.3528825  0.46448255 0.03366041 0.68467236
 0.74011743 0.8724445  0.22632635 0.22319686 0.3103881  0.7223358
 0.13318717 0.5480639  0.5746088  0.8996835  0.00946367 0.5212307
 0.6345445  0.1993283  0.72942245 0.54583454 0.10756552 0.6767061
 0.6602763  0.33695042 0.60141766 0.2106257

####One-hot encoding tensors

In [125]:
#creating a list of indices
some_list = [0,1,2,3] #wrt red, green, blue, yellow (say)
#One hot encode our listof indices
tf.one_hot(some_list, depth = 4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [126]:
#specifying custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value="ON", off_value="OFF")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'ON', b'OFF', b'OFF', b'OFF'],
       [b'OFF', b'ON', b'OFF', b'OFF'],
       [b'OFF', b'OFF', b'ON', b'OFF'],
       [b'OFF', b'OFF', b'OFF', b'ON']], dtype=object)>

### Squaring, log, squareroot ###

In [135]:
#creating a new tensor
H= tf.range(1, 10)
H, H.dtype, H.ndim

(<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9])>,
 tf.int32,
 1)

In [136]:
#squaring it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81])>

In [137]:
#finding square root (method required non-int dtype)
tf.sqrt(tf.cast(H, dtype=tf.float16))

<tf.Tensor: shape=(9,), dtype=float16, numpy=
array([1.   , 1.414, 1.732, 2.   , 2.236, 2.45 , 2.646, 2.828, 3.   ],
      dtype=float16)>

In [140]:
#finding log (method required non-int dtype)
tf.math.log(tf.cast(H, dtype=tf.float16))

<tf.Tensor: shape=(9,), dtype=float16, numpy=
array([0.    , 0.6934, 1.099 , 1.387 , 1.609 , 1.792 , 1.946 , 2.08  ,
       2.197 ], dtype=float16)>

#### Tensors and NumPy ####

Tensorflow interacts beautifully with NumPy array.

*Note:*
One of the main differences between a TensorFLow tensor and a NumPy array is that a tensorflow tensor can be run on a GPU or TPU (for faster numerical processing)

In [149]:
#creating tensor directly form NumPy array
J = tf.constant(np.array([1,2,3,4], dtype=np.float16))
J

<tf.Tensor: shape=(4,), dtype=float16, numpy=array([1., 2., 3., 4.], dtype=float16)>

In [150]:
#Convert the tensor back to NumPy array
np.array(J), type(np.array(J)), J.numpy(), type(J.numpy()), J.numpy()[0]

(array([1., 2., 3., 4.], dtype=float16),
 numpy.ndarray,
 array([1., 2., 3., 4.], dtype=float16),
 numpy.ndarray,
 1.0)

In [152]:
#the default type of each atre slightly different
numpy_J = tf.constant(np.array([1.,2.,3.,4.]))
tensor_J = tf.constant([1.,2.,3.,4.])
type(numpy_J), numpy_J.dtype, type(tensor_J), tensor_J.dtype

(tensorflow.python.framework.ops.EagerTensor,
 tf.float64,
 tensorflow.python.framework.ops.EagerTensor,
 tf.float32)

#### Finding access to GPUs ####

In [153]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [154]:
!nvidia-smi

Thu Feb  3 21:27:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.65       Driver Version: 511.65       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   49C    P8     1W /  N/A |   2318MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------